In [ ]:
%%bash -x

# ============================================
# 0. CLEAN EVERYTHING
# ============================================
rm -rf /content/py310
rm -rf /content/ProbPose
rm -rf /content/chumpy-fixed

# ============================================
# 1. Install Python 3.10
# ============================================
add-apt-repository ppa:deadsnakes/ppa -y
apt update -y
apt install -y python3.10 python3.10-dev python3.10-distutils python3.10-venv
curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# ============================================
# 2. Create venv
# ============================================
python3.10 -m venv /content/py310
/content/py310/bin/python -m pip install --upgrade pip setuptools wheel

# ============================================
# 3. INSTALL NUMPY 1.26 FIRST (LOCK IT)
# ============================================
/content/py310/bin/python -m pip install numpy==1.26.4
echo "numpy<2" > /content/py310/constraints.txt

# ============================================
# 4. Install PyTorch 2.0.1 + cu118
# ============================================
/content/py310/bin/python -m pip install torch==2.0.1+cu118 \
    torchvision==0.15.2+cu118 \
    --extra-index-url https://download.pytorch.org/whl/cu118 \
    -c /content/py310/constraints.txt

# ============================================
# 5. Install MMCV + mmengine + mmdet + mmpretrain
# ============================================
/content/py310/bin/python -m pip install -U openmim -c /content/py310/constraints.txt
/content/py310/bin/mim install mmengine -c /content/py310/constraints.txt
/content/py310/bin/mim install "mmcv==2.1.0" -c /content/py310/constraints.txt
/content/py310/bin/mim install "mmdet==3.3.0" -c /content/py310/constraints.txt
/content/py310/bin/mim install "mmpretrain==1.2.0" -c /content/py310/constraints.txt

# ============================================
# 6. Local chumpy (works with NumPy 1.26)
# ============================================
mkdir -p /content/chumpy-fixed/chumpy

cat << 'EOF' > /content/chumpy-fixed/chumpy/__init__.py
from .ch import Ch
from .utils import *
EOF

cat << 'EOF' > /content/chumpy-fixed/chumpy/ch.py
import numpy as np

class Ch(np.ndarray):
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def compute_r(self):
        return np.asarray(self)

    def compute_dr_wrt(self, wrt):
        return None
EOF

cat << 'EOF' > /content/chumpy-fixed/chumpy/utils.py
import numpy as np

def row(x):
    return np.asarray(x).reshape(1, -1)

def col(x):
    return np.asarray(x).reshape(-1, 1)
EOF

cat << 'EOF' > /content/chumpy-fixed/setup.py
from setuptools import setup, find_packages
setup(
    name='chumpy',
    version='0.1.0',
    packages=find_packages(),
    install_requires=['numpy<2']
)
EOF

/content/py310/bin/python -m pip install /content/chumpy-fixed -c /content/py310/constraints.txt

# ============================================
# 7. Clone ProbPose
# ============================================
git clone https://github.com/mirapurkrabek/ProbPose_code.git /content/ProbPose

# ============================================
# 8. Install ProbPose deps
# ============================================
cd /content/ProbPose
/content/py310/bin/python -m pip install -r requirements.txt --no-deps -c /content/py310/constraints.txt
/content/py310/bin/python -m pip install -e . -c /content/py310/constraints.txt

# ============================================
# 9. Download weights
# ============================================
cd /content
wget -O ProbPose-s.pth "https://huggingface.co/vrg-prague/ProbPose-s/resolve/main/ProbPose-s.pth"

echo "==============================================="
echo "ProbPose setup COMPLETED with NumPy LOCKED."
echo "==============================================="


In [ ]:
!MPLBACKEND=Agg /content/py310/bin/python /content/ProbPose/demo/image_demo.py \
    /content/ProbPose/demo/resources/CropCOCO_single_example.jpg \
    /content/ProbPose/configs/body_2d_keypoint/topdown_probmap/coco/td-pm_ProbPose-small_8xb64-210e_coco-256x192.py \
    /content/ProbPose-s.pth \
    --out-file /content/result.jpg \
    --draw-heatmap


# 🎥 Video to 2D Keypoints CSV Extraction

This section allows you to extract 2D keypoints from videos and save them to CSV format.

The CSV will contain:
- **Frame numbers** (0-indexed)
- **17 COCO body keypoints** per frame
- **X, Y coordinates and confidence scores** for each keypoint

Body keypoints: nose, eyes, ears, shoulders, elbows, wrists, hips, knees, ankles

## Option 1: Upload Your Own Video

In [ ]:
from google.colab import files
import os

# Upload your video file
print("📤 Please upload your video file:")
uploaded = files.upload()

# Get the uploaded filename
video_filename = list(uploaded.keys())[0]
video_path = f'/content/{video_filename}'

print(f"\n✅ Video uploaded: {video_filename}")

In [ ]:
# Extract keypoints from uploaded video
print("📝 Extracting 2D keypoints...")

!MPLBACKEND=Agg /content/py310/bin/python /content/video_to_2d_keypoints.py \
    {video_path} \
    --output /content/keypoints_2d.csv \
    --config /content/ProbPose/configs/body_2d_keypoint/topdown_probmap/coco/td-pm_ProbPose-small_8xb64-210e_coco-256x192.py \
    --weights /content/ProbPose-s.pth \
    --device cuda

print("\n✅ Extraction complete!")

## Option 2: Use Video from URL

In [ ]:
# Download video from URL
# Replace with your video URL
video_url = "https://storage.googleapis.com/gtv-videos-bucket/sample/ForBiggerEscapes.mp4"

print(f"📥 Downloading video from URL...")
!wget -O /content/input_video.mp4 "{video_url}"

print("\n📝 Extracting 2D keypoints...")
!MPLBACKEND=Agg /content/py310/bin/python /content/video_to_2d_keypoints.py \
    /content/input_video.mp4 \
    --output /content/keypoints_2d.csv \
    --config /content/ProbPose/configs/body_2d_keypoint/topdown_probmap/coco/td-pm_ProbPose-small_8xb64-210e_coco-256x192.py \
    --weights /content/ProbPose-s.pth \
    --device cuda

print("\n✅ Extraction complete!")

## Preview and Download CSV Results

In [ ]:
import pandas as pd

# Load and preview the CSV
df = pd.read_csv('/content/keypoints_2d.csv')

print("📊 CSV Preview:")
print(f"Total frames: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nColumn names:")
print(df.columns.tolist()[:10], "...")  # Show first 10 columns

print(f"\n\nFirst 5 rows:")
print(df.head())

print(f"\n\nSample keypoints from frame 0:")
frame_0 = df.iloc[0]
print(f"  Nose: ({frame_0['nose_x']:.1f}, {frame_0['nose_y']:.1f}), confidence: {frame_0['nose_conf']:.3f}")
print(f"  Left shoulder: ({frame_0['left_shoulder_x']:.1f}, {frame_0['left_shoulder_y']:.1f}), confidence: {frame_0['left_shoulder_conf']:.3f}")
print(f"  Right shoulder: ({frame_0['right_shoulder_x']:.1f}, {frame_0['right_shoulder_y']:.1f}), confidence: {frame_0['right_shoulder_conf']:.3f}")
print(f"  Left hip: ({frame_0['left_hip_x']:.1f}, {frame_0['left_hip_y']:.1f}), confidence: {frame_0['left_hip_conf']:.3f}")
print(f"  Right hip: ({frame_0['right_hip_x']:.1f}, {frame_0['right_hip_y']:.1f}), confidence: {frame_0['right_hip_conf']:.3f}")

In [ ]:
from google.colab import files

# Download the generated CSV file
print("📥 Downloading keypoints CSV...")
files.download('/content/keypoints_2d.csv')

print("✅ Download complete!")

In [ ]:
## 📋 CSV Format Information

The generated CSV contains:

### Columns (52 total):
- `frame` - Frame number (0-indexed)
- For each of 17 keypoints: `{keypoint}_x`, `{keypoint}_y`, `{keypoint}_conf`

### Body Keypoints (COCO format):
1. nose
2. left_eye, right_eye
3. left_ear, right_ear
4. left_shoulder, right_shoulder
5. left_elbow, right_elbow
6. left_wrist, right_wrist
7. left_hip, right_hip
8. left_knee, right_knee
9. left_ankle, right_ankle

### Example Usage in Python:
```python
import pandas as pd

# Load CSV
df = pd.read_csv('keypoints_2d.csv')

# Get specific frame
frame_10 = df[df['frame'] == 10]
nose_x = frame_10['nose_x'].values[0]
nose_y = frame_10['nose_y'].values[0]

# Filter by confidence
high_conf_frames = df[df['nose_conf'] > 0.9]
```